## Importando bibliotecas

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from ucimlrepo import fetch_ucirepo

## Tratamento dos dados

In [2]:
dataset = fetch_ucirepo(id=45)
df = pd.DataFrame(dataset.data.features)
if dataset.data.targets is not None:
    # targets pode ser Series ou DataFrame dependendo do conjunto
    targets = pd.DataFrame(dataset.data.targets)
    df = pd.concat([df, targets], axis=1)

df = df.dropna()
labels = df["num"].values
df.drop("num", axis=1, inplace=True)
dados = df.to_numpy()

## Método de agrupamento

In [3]:
class MFCM():
    def __init__(self, c, X, m):
        self.c = c
        self.n = X.shape[0]
        self.p = X.shape[1]
        self.m = m

        ##
        self.global_var = np.var(X, axis=0).mean() # pré-calcula a variância global dos dados para regularização

    def initialize_u(self):
        u_flat = np.random.dirichlet(alpha=np.ones(self.c * self.p), size=self.n)
        return u_flat.reshape(self.n, self.c, self.p)
    
    def initialize_lambda(self):
        return np.ones((self.c, self.p))
    
    def find_centroides(self, X, U):
        u_m = U ** self.m
        numerador = np.sum(u_m * X[:, np.newaxis, :], axis=0)
        denominador = np.sum(u_m, axis=0)
        denominador = np.fmax(denominador, np.finfo(np.float64).eps)
        return numerador / denominador
    
    def get_distances(self, X, V):
        return (X[:, np.newaxis, :] - V[np.newaxis, :, :]) ** 2

    def update_u(self, D, Lambda):
        power = 1.0 / (self.m - 1)
        eps = np.finfo(np.float64).eps
        
        weighted_dist = D * Lambda
        weighted_dist = np.fmax(weighted_dist, eps) 
        
        term = weighted_dist ** (-power)
        
        denominator = np.sum(term, axis=(1, 2), keepdims=True)
        denominator = np.fmax(denominator, eps)
        
        return term / denominator
    
    def update_lambda(self, D, U):
        eps = np.finfo(np.float64).eps
        
        term_k = (U ** self.m) * np.fmax(D, eps)
        S_ij = np.sum(term_k, axis=0) 
        
        # Regularização Aditiva
        ##
        regularization_factor = 0.01 * self.global_var
        S_ij = S_ij + regularization_factor 
        
        prod_S = np.prod(S_ij, axis=1, keepdims=True)
        numerator = prod_S ** (1.0 / self.p)
        
        Lambda = numerator / S_ij
        
        return Lambda

    ##
    def calculate_objective_function(self, D, U, Lambda):
        term = (U ** self.m) * D
        sum_k = np.sum(term, axis=0)
        return np.sum(Lambda * sum_k)

## Clustering

In [4]:
def mfcm_run(dados, num_clusters, m=2, max_iter=1000, epsilon=1e-6):
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(dados)
    
    mfcm = MFCM(c=num_clusters, X=X_scaled, m=m)
    
    indices = np.random.choice(X_scaled.shape[0], num_clusters, replace=False)
    centroids = X_scaled[indices]
    
    Lambda = np.ones((num_clusters, X_scaled.shape[1]))
    
    D = mfcm.get_distances(X_scaled, centroids)
    U = mfcm.update_u(D, Lambda)
    
    WARM_UP_ITERS = 20 

    for i in range(max_iter):
        U_old = U.copy()
        
        centroids = mfcm.find_centroides(X_scaled, U)
        D = mfcm.get_distances(X_scaled, centroids)
        
        ##
        if i >= WARM_UP_ITERS:
            Lambda = mfcm.update_lambda(D, U) 
        
        U = mfcm.update_u(D, Lambda)
        
        if np.linalg.norm(U - U_old) < epsilon:
            break
        
    J_final = mfcm.calculate_objective_function(D, U, Lambda)
    Delta = np.sum(U, axis=2)
    
    return centroids, U, Delta, Lambda, J_final

## Simulação de Monte Carlo

In [5]:
def calculate_objective_function(self, D, U, Lambda):
        term = (U ** self.m) * D
        sum_k = np.sum(term, axis=0)
        weighted_sum = Lambda * sum_k
        J = np.sum(weighted_sum)
        return J

In [9]:
def run_final_experiment(dados, labels, num_clusters=3, num_trials=100, restarts=50):
    results_ari = []
    results_ami = []
    
    for t in range(num_trials):
        best_J = np.inf
        best_pred = None
        
        for r in range(restarts):
            try:
                centroids, U, Delta, Lambda, J = mfcm_run(dados, num_clusters, m=2)
                
                if J > 1e-3 and J < best_J:
                    best_J = J
                    best_pred = np.argmax(Delta, axis=1)
            except:
                continue
        
        if best_pred is not None:
            ari = adjusted_rand_score(labels, best_pred)
            ami = adjusted_mutual_info_score(labels, best_pred)
            
            results_ari.append(ari)
            results_ami.append(ami)
            
            print(f"Trial {t+1}: J={best_J:.4f} | ARI={ari:.4f} | AMI={ami:.4f}")
        else:
            print(f"Trial {t+1}: Falha (Singularidade em todas as tentativas).")

    print(f"Mean ARI: {np.mean(results_ari):.4f} +/- {np.std(results_ari):.4f}")
    print(f"Mean AMI: {np.mean(results_ami):.4f} +/- {np.std(results_ami):.4f}")

In [ ]:
run_final_experiment(dados, labels, num_clusters=5, num_trials=100, restarts=50)